<a href="https://colab.research.google.com/github/EiPhyuSinn/Retrieval-QA-Langchain-with-togetherAI-API/blob/main/qa_togetherai_mixtral_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain langchain-community


In [ ]:
pip install -U langchain-openai


In [ ]:
from langchain_openai import ChatOpenAI
import os
from langchain_core.messages import HumanMessage

os.environ["TOGETHER_API_KEY"] = "your_api_key"

llm = ChatOpenAI(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    openai_api_key=os.environ["TOGETHER_API_KEY"],
    openai_api_base="https://api.together.xyz/v1",
)

response = llm.invoke([HumanMessage(content="What's the difference between AI and machine learning?")])
print(response.content)

In [ ]:
! pip install sentence-transformers


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector = embedding.embed_query("Hello, how are you?")


In [ ]:
len(vector)

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='Mental_Health_FAQ.csv',source_column='Questions')
data = loader.load()
data[:3]

In [ ]:
pip install faiss-cpu

In [ ]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=data,
                          embedding=embedding)
retriever = db.as_retriever(score_threshold = 0.7)
retriever

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = """
You are provided with some context and a question. Answer the question **only** using the given context.
Focus on extracting as much detail as possible from the "response" section in the document.
If the context does not contain the answer, simply reply with "I don't know." Do not make up answers.

Context:
{context}

Question:
{question}
"""

PROMPT = PromptTemplate(
    template=custom_prompt,
    input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
answer1 = qa_chain("What does it mean to have a mental illness?")

In [ ]:
answer1.keys()

In [ ]:
answer1['query']

In [ ]:
answer1['result']

In [ ]:
answer1['source_documents']